University of Helsinki, Master's Programme in Mathematics and Statistics  
MAST32001 Computational Statistics, Autumn 2022  
Luigi Acerbi  

# Week 2 exercises

## 1. Permutation testing (6 pts)

We will use permutation testing to study if the mother's age (`age`) affects the birth weight (`bwt`) of their babies. We will use the absolute difference in the means as the test statistic. We will focus the analysis on full term pregnancies (`gestation >= 273`).

*Note*: When reporting a $p$-value for $b$ more extreme tests out of $m$, use $p = (b+1)/(m+1)$ to avoid zero p-values. 50000 permutations will be sufficient for obtaining the required accuracy.

1. Load the data set below. Test whether the birth weights (`bwt`) of babies with young (`age < 26`) and older (`age >= 26`) mothers are statistically significantly different using the difference of the means as the test statistic. Report the $p$-value you obtain in Moodle.
2. Stratify the analysis by the variable smoking status of the mothers by splitting to separate smoker (`smoke = 0`) and non-smoker (`smoke = 1`) groups. Constrain the permutations so that only changes within each group are allowed. After the permutation, merge the two groups back together to compute the means. Report the $p$-value you obtain in Moodle.

In [1]:
import pandas as pd
import numpy as np
import numpy.random as npr

# Load the data set
babies_full = pd.read_csv("https://www2.helsinki.fi/sites/default/files/atoms/files/babies2.txt", sep='\t')

# Pick a subset
babies1 = babies_full.iloc[(babies_full['gestation']>=273).values]

def shuffle(x1, x2):
    """Return a random reshuffling of elements in two arrays"""
    n1 = len(x1)
    z = npr.permutation(np.concatenate((x1, x2)))
    return z[0:n1], z[n1:]

def merge(x1, x2):
    """Merge two data sets"""
    return np.concatenate((x1, x2))

def permutation_test_absmeandiff(x1, x2, N_perm):
    """Perform permutation test for the absolute mean difference of two groups."""
    truediff = np.abs(np.mean(x1) - np.mean(x2))
    meandiffs = np.zeros(N_perm)
    for i in range(N_perm):
        z1, z2 = shuffle(x1, x2)
        meandiffs[i] = np.abs(np.mean(z1) - np.mean(z2))
    return (np.sum(truediff <= meandiffs)+1)/(len(meandiffs)+1)

def permutation_test_stratified_absmeandiff(x1a, x1b, x2a, x2b, N_perm):
    """Perform permutation test for the absolute mean difference of two groups with stratification."""
    truediff = np.abs(np.mean(merge(x1a, x1b)) - np.mean(merge(x2a, x2b)))
    meandiffs = np.zeros(N_perm)
    for i in range(N_perm):
        # Shuffle two subgroups independently
        z1a, z2a = shuffle(x1a, x2a)
        z1b, z2b = shuffle(x1b, x2b)
        # Re-merge the two subgroups for each unit
        z1 = merge(z1a, z1b)
        z2 = merge(z2a, z2b)
        # Compute the difference as usual
        meandiffs[i] = np.abs(np.mean(z1) - np.mean(z2))
    return (np.sum(truediff <= meandiffs)+1)/(len(meandiffs)+1)

### Part 1
Iyounger = ( babies1['age'] < 26 ).values
Iolder = ( babies1['age'] >= 26 ).values

younger_vals = babies1['bwt'].values[Iyounger]
older_vals = babies1['bwt'].values[Iolder]

npr.seed(0)
pval = permutation_test_absmeandiff(younger_vals, older_vals, 50000)
print("p-value without stratification = ", pval)

p-value without stratification =  0.0751584968300634


In [2]:
Iyounger = ( babies1['age'] < 26 ).values
Iolder = ( babies1['age'] >= 26 ).values
smoker = ( babies1['smoke'] == 1 ).values
non_smoker = ( babies1['smoke'] == 0 ).values

younger_smoker = babies1['bwt'].values[Iyounger & smoker]
younger_non_smoker = babies1['bwt'].values[Iyounger & non_smoker]
older_smoker = babies1['bwt'].values[Iolder & smoker]
older_non_smoker = babies1['bwt'].values[Iolder & non_smoker]

npr.seed(0)
pval = permutation_test_stratified_absmeandiff(younger_smoker, younger_non_smoker, \
                                               older_smoker, older_non_smoker, 50000)
print("p-value with stratification = ", pval)

p-value with stratification =  0.07897842043159137


## 2. Bootstrap confidence intervals on data statistics (4 pts)

In this exercise we use bootstrap to estimate confidence intervals for various quantities. (Using 1000 bootstrap samples will give you enough accuracy assuming everything is correctly done.)

1. Use bootstrap to estimate the central 95% confidence interval for the mean of `bwt` in the *full* data set loaded in Problem 1. Report the lower and upper ends of the interval in Moodle.
2. Use bootstrap to estimate the central 95% confidence interval for the mean of `bwt` in the smaller subset (`gestation >= 273`) of the data set used in Problem 1. Report the lower and upper ends of the interval in Moodle.
3. Use bootstrap to estimate the central 95% confidence interval for the correlation coefficient of `gestation` and `age` in the full data set loaded in Problem 1. What does this tell about the relation between the duration of the pregnancy and the age of the mother? Report the bounds of the interval in Moodle.
4. Use bootstrap to estimate the central 95% confidence interval for the correlation coefficient of `gestation` and `age` in the smaller subset (`gestation >= 273`) used in Problem 1. What does this tell about the relation between the duration of the pregnancy and the age of the mother? Report the bounds of the interval in Moodle.

*Hint*: Remember that the size of the bootstrap sample is always the same as the size of the original data set.

In [3]:
npr.seed(0)
bwt = babies_full['bwt']
n_bootstrap = 1000
n = len(bwt)
bootstrap_means = np.array([np.mean(npr.choice(bwt, replace=True, size=n)) for i in range(n_bootstrap)])
print('1. bootstrap interval:', np.percentile(bootstrap_means, [2.5, 97.5]))

bwt_gest = babies1['bwt']
n_bootstrap = 1000
n = len(bwt_gest)
bootstrap_means = np.array([np.mean(npr.choice(bwt_gest, replace=True, size=n)) for i in range(n_bootstrap)])
print('2. bootstrap interval:', np.percentile(bootstrap_means, [2.5, 97.5]))

n_bootstrap = 1000
corr_coefs = np.zeros(n_bootstrap)
n = len(babies_full)
for i in range(n_bootstrap):
    sample = babies_full.sample(n=n, replace=True)
    corr_coefs[i] = np.corrcoef( sample['gestation'], sample['age'] )[0,1]
print('3. bootstrap interval:', np.percentile(corr_coefs, [2.5, 97.5]))

n_bootstrap = 1000
corr_coefs = np.zeros(n_bootstrap)
n = len(babies1)
for i in range(n_bootstrap):
    sample = babies1.sample(n=n, replace=True)
    corr_coefs[i] = np.corrcoef( sample['gestation'], sample['age'] )[0,1]
print('4. bootstrap interval:', np.percentile(corr_coefs, [2.5, 97.5]))

1. bootstrap interval: [118.34590793 120.50758738]
2. bootstrap interval: [122.52281609 124.64827586]
3. bootstrap interval: [-0.1097222   0.00104757]
4. bootstrap interval: [-0.09141982  0.04007667]


## 3. Bootstrap confidence intervals on parameter estimates (4 pts)

In this task, we will use bootstrap to obtain confidence intervals on maximum likelihood parameter estimates for linear regression models. We will apply simple case resampling, i.e. resampling the individuals and then fitting the model using the data $(x_i, y_i)$ from these individuals. There are alternative methods that may work better when the data are limited, but in our case there are enough observations so that this will not be a problem. 1000 bootstrap samples will again give you enough accuracy.

A linear regression fit to scalar $x_i, y_i$ involves fitting the model
$$ y_i = \alpha + \beta x_i + \epsilon_i, $$
where $\beta$ is the regression coefficient and $\alpha$ is the intercept. Assuming $\epsilon_i \sim N(0, \sigma^2)$, the log-likelihood of the model is
$$ \log p(Y | X, \alpha, \beta) = \sum_{i=1}^n \log p(y_i | x_i, \alpha, \beta)
  = \sum_{i=1}^n - \frac{1}{2 \sigma^2} (y_i - \alpha - \beta x_i)^2 + C, $$
where $C$ is independent of $\alpha, \beta$. This is maximised when
$$ \hat{\beta}= \frac{\sum_{i = 1}^n (x_i - \bar{x})(y_i - \bar{y}) }{ \sum_{i = 1}^n (x_i - \bar{x})^2} \\
   \hat{\alpha} = \bar{y} - \hat{\beta} \bar{x},$$
where $\bar{x} = \frac{1}{n} \sum_{i = 1}^n x_i$ and $\bar{y} = \frac{1}{n} \sum_{i = 1}^n y_i$.

1. Implement the above linear regression model to predict `gestation` ($y$) as a function of `age` ($x$) in the full data set. Report the estimated $\beta$ in Moodle.
2. Use bootstrap to estimate the confidence interval of the regression coefficient $\beta$ in the above model by resampling the individuals used to fit the model. Report the lower and upper bounds of the central 95% confidence interval of $\beta$ in Moodle.


In [4]:
import pandas as pd
import numpy as np
import numpy.random as npr

# Load the data set
babies_full = pd.read_csv("https://www2.helsinki.fi/sites/default/files/atoms/files/babies2.txt", sep='\t')

babies3 = babies_full

def estimator(x, y):
    n = len(x)
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    num = 0
    den = 0
    for i in range(n):
        num += (x[i] - x_mean)*(y[i] - y_mean)
        den += (x[i] - x_mean)**2
    beta = num / den
#     alpha = y_mean - beta*x_mean
    return beta

beta = estimator(babies3['age'].values, babies3['gestation'].values)
print("Estimated beta: ", beta)

n_bootstrap = 1000
betas = np.zeros(n_bootstrap)
n = len(babies3)
for i in range(n_bootstrap):
    sample = babies3.sample(n=n, replace=True)
    betas[i] = estimator( sample['age'].values, sample['gestation'].values )
print('bootstrap interval:', np.percentile(betas, [2.5, 97.5]))

Estimated beta:  -0.14772779151120366
bootstrap interval: [-0.30191935  0.01800218]


## 4. Density estimation (6 pts)

1. Estimate the joint density of `bwt` and `age` in the full data set using kernel density estimation with a 2-dimensional Gaussian kernel
$$ K(\mathbf{x}) = \frac{1}{2\pi} \exp\left( - \frac{\|\mathbf{x}\|^2}{2} \right)
 = \frac{1}{2\pi} \exp\left( - \frac{x_1^2 + x_2^2}{2} \right) $$
using bandwidth $h=5$. Report the value of the estimated density at point `bwt=110`, `age=31` in Moodle.

*Hint*: you can verify your results by ploting a 2D histogram of the data (`matplotlib.pyplot.hist2d`) and a contour plot of the estimated density (see e.g. Sec. 5.1.1 in Course notes for a contour plot example).

2. With the Gaussian kernel above, use leave-one-out (LOO) cross validation to find the optimal $h$ in the range `np.linspace(1.0, 5.0, 50)`. The optimal $h$ maximizes the LOO log-likelihood. Report the value of $h$ and the value of the estimated density at `bwt=110`, `age=31` in Moodle.

3. Use $k$-fold cross validation with $k=17$ to find the optimal $h$ in the range `np.linspace(1.0, 5.0, 50)`. Report the value of $h$ and the value of the estimated density at `bwt=110`, `age=31` in Moodle. For this exercise, the sample point indices for the $k$ partitions of the data should consist of consecutive indices, e.g. the first partition should be the data with indices `0:69` and so on. (In practical applications it is generally good practice to randomly permute the indices as well when creating partitions, but don't do it for this exercise.)

In [5]:
def K_mvgauss(x):
    """Multivariate normal kernel."""
    return 1/(2*np.pi) * np.exp(-0.5*np.sum(x**2, 1))

def kernel_density_KCV(t, x, h):
    y = np.zeros(len(t))
    d = x.shape[1]
    for i in range(len(t)):
        y[i] = np.mean(K_mvgauss((t[i] - x)/ h)) / h**d
    return y

def mv_kernel_density(t, x, h):
    """Multivariate normal kernel density estimate."""
    d = x.shape[1]
    return np.mean(K_mvgauss((t - x)/h))/h**d

### Part 1
data = babies_full[['bwt', 'age']].values
observation = np.array([110, 31])
h = 5

print('Estimated density at bwt=110, age=31: ', mv_kernel_density(observation, data, h))

### Part 2
def mv_loocv_kernel_density(x, hs):
    """Multivariate kernel density estimation via leave-one-out cross-validation """
    logls = np.zeros(len(hs))
    for j in range(len(x)):
        train_x = np.delete(x, j, axis=0) # Remove the j-th point
        test_x = np.array([x[j]]) # Single test point
        for i in range(len(hs)):
            logls[i] += np.log(mv_kernel_density(test_x, train_x, hs[i]))
    # print('LOO log-probabilities:', logls)
    return hs[np.argmax(logls)]

hs = np.linspace(1.0, 5.0, 50)
optimal_h = mv_loocv_kernel_density(data, hs)
estimate_opt_h = mv_kernel_density(observation, data, optimal_h)
print(f'Estimated density with optimal LOOCV h={optimal_h} at bwt=110, age=31 is: {estimate_opt_h}')

### Part 3
npr.seed(0)
def k_foldcv(x, k, hs):
    """K-fold cross-validation."""
    N = len(x)
    I = npr.permutation(N)
    logls = np.zeros(len(hs))
    for j in range(k):
        testI = np.zeros(N, bool)
        testI[((j*N)//k):(((j+1)*N)//k)] = True
        trainI = ~testI
        for i in range(len(hs)):
            logls[i] += np.sum(np.log(kernel_density_KCV(x[I[testI]], x[I[trainI]], hs[i])))
    return (hs, logls)

k = 17
hs, logls = k_foldcv(data, k, hs)
opt_k_h = hs[np.argmax(logls)]
estimate_k_opt_h = mv_kernel_density(observation, data, opt_k_h)
print(f'Estimated density with K-fold CV h={opt_k_h} at bwt=110, age=31 is: {estimate_k_opt_h}')

Estimated density at bwt=110, age=31:  0.0007101226311425995
Estimated density with optimal LOOCV h=2.3061224489795915 at bwt=110, age=31 is: 0.0005836157630220638
Estimated density with K-fold CV h=2.387755102040816 at bwt=110, age=31 is: 0.000592793787708447
